In [1]:
# ============================================================
# 0) OS / paths (optional)
# ============================================================
import os
os.environ["R_HOME"] = "/home/kgr851/.conda/envs/bulkATAC_RNA/lib/R"

# ============================================================
# 1) Warnings / display defaults
# ============================================================
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# ============================================================
# 2) Core Python libs
# ============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================
# 3) Plotting style
# ============================================================
sns.set_context("talk")
sns.set(rc={"figure.figsize": (4, 3.5), "figure.dpi": 200})
sns.set_style("whitegrid")
plt.rcParams["pdf.fonttype"] = 42  # editable text in Illustrator

# ============================================================
# 4) R in Jupyter (rpy2)
# ============================================================
%load_ext rpy2.ipython

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# Use the modern context-based approach to avoid deprecation warning
# This replaces the deprecated pandas2ri.activate() call

import pandas as pd
import numpy as np
import re
from pathlib import Path

# load counts, drop gene-name column, build meta, write CSVs

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- Paths ---
SALMON_DIR = Path("/datasets/renew_kirkeby/gaurav/PIPELINE-nfcore-rnaseq/GRCh38/salmon")
COUNTS_TSV = SALMON_DIR / "salmon.merged.gene_counts.tsv"

OUTDIR = Path("/home/kgr851/BULK_RNA")
OUTDIR.mkdir(parents=True, exist_ok=True)

print("Reading:", COUNTS_TSV)

# 1) Read merged counts (gene_id already as index)
counts = pd.read_csv(COUNTS_TSV, sep="\t", index_col=0)
print("Raw counts shape:", counts.shape)

# 2) Drop non-numeric columns (e.g. gene_symbol like 'TSPAN6')
counts_num = counts.apply(pd.to_numeric, errors="coerce")

bad_cols = counts_num.columns[counts_num.isna().all(axis=0)].tolist()
if bad_cols:
    print("Dropping non-numeric columns:", bad_cols)
    counts_num = counts_num.drop(columns=bad_cols)

# Any remaining NA → 0
counts_num = counts_num.fillna(0)

# 3) Convert to integer counts for DESeq2
counts_int = counts_num.round(0).astype(int)

print("Numeric counts shape:", counts_int.shape)
print("Example genes:", counts_int.index[:3].tolist())
print("Example samples:", counts_int.columns[:5].tolist())

# 4) Build metadata (v2 style: parse from sample names)
def parse_sample(sample):
    toks = sample.split("_")
    cellline = toks[0]
    time = toks[1]
    replicate = toks[-1].replace("Rep", "")
    fate = "_".join(toks[2:-1])
    return cellline, time, fate, replicate

meta = pd.DataFrame(
    [parse_sample(s) for s in counts_int.columns],
    columns=["cellline", "time", "fate", "replicate"],
)
meta.insert(0, "sample", counts_int.columns)

# 5) Write outputs for R cell
counts_out = OUTDIR / "RNA_counts_all_INT.csv"
meta_out   = OUTDIR / "RNA_metadata_all.csv"

counts_int.to_csv(counts_out)
meta.to_csv(meta_out, index=False)

print("Wrote:", counts_out)
print("Wrote:", meta_out)

# 6) Quick sanity
print("\nMeta head:")
print(meta.head())

print("\nCounts per group:")
print(
    meta.groupby(["cellline", "time", "fate"])
        .size()
        .sort_values(ascending=False)
        .head(15)
)

Reading: /datasets/renew_kirkeby/gaurav/PIPELINE-nfcore-rnaseq/GRCh38/salmon/salmon.merged.gene_counts.tsv


Raw counts shape: (62812, 65)
Dropping non-numeric columns: ['gene_name']
Numeric counts shape: (62812, 64)
Example genes: ['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419']
Example samples: ['H9_0h_undiff_Rep1', 'H9_0h_undiff_Rep2', 'H9_12h_FB_Rep1', 'H9_12h_FB_Rep2', 'H9_12h_HB_Rep1']
Wrote: /home/kgr851/BULK_RNA/RNA_counts_all_INT.csv
Wrote: /home/kgr851/BULK_RNA/RNA_metadata_all.csv

Meta head:
              sample cellline time    fate replicate
0  H9_0h_undiff_Rep1       H9   0h  undiff         1
1  H9_0h_undiff_Rep2       H9   0h  undiff         2
2     H9_12h_FB_Rep1       H9  12h      FB         1
3     H9_12h_FB_Rep2       H9  12h      FB         2
4     H9_12h_HB_Rep1       H9  12h      HB         1

Counts per group:
cellline  time  fate  
H9        0h    undiff    2
          12h   FB        2
                HB        2
                MB        2
          24h   FB        2
                HB        2
                MB        2
          3d    FB        2
       

In [3]:
%%R
suppressPackageStartupMessages({
  library(DESeq2)
  library(dplyr)
  library(tibble)
})

outdir <- "/home/kgr851/BULK_RNA"

# 1) Read inputs written by Python Cell 1
counts <- read.csv(file.path(outdir, "RNA_counts_all_INT.csv"),
                   row.names = 1, check.names = FALSE)

meta <- read.csv(file.path(outdir, "RNA_metadata_all.csv"),
                 stringsAsFactors = FALSE)
rownames(meta) <- meta$sample

# 2) Ensure ordering matches (counts columns must match meta rownames)
counts <- counts[, rownames(meta)]

cat("counts dim:", dim(counts), "\n")
cat("meta dim:", dim(meta), "\n")

# Quick sanity: show available fate labels
cat("\nUnique fates:\n")
print(sort(unique(meta$fate)))

# 3) DESeq2 contrast runner (writes full + TOP50)
run_contrast <- function(cellline, time, fateA, fateB, tag) {

  keep <- meta$cellline == cellline &
          meta$time == time &
          meta$fate %in% c(fateA, fateB)

  meta2 <- meta[keep, , drop = FALSE]
  cts2  <- counts[, rownames(meta2), drop = FALSE]

  cat("\n---", tag, "---\n")
  cat("Subset n samples:", ncol(cts2), "\n")
  print(table(meta2$fate))

  if (ncol(cts2) < 4) {
    stop(paste("Too few samples:", cellline, time, fateA, "vs", fateB, "n=", ncol(cts2)))
  }

  meta2$fate <- factor(meta2$fate)
  meta2$fate <- relevel(meta2$fate, ref = fateA)

  dds <- DESeqDataSetFromMatrix(countData = cts2, colData = meta2, design = ~ fate)
  dds <- dds[rowSums(counts(dds)) >= 10, ]
  dds <- DESeq(dds, quiet = TRUE)

  #res <- results(dds, contrast = c("fate", fateB, fateA)) %>%
  res <- results(dds, contrast = c("fate", fateA, fateB)) %>%
    as.data.frame() %>%
    rownames_to_column("gene_id") %>%
    arrange(padj)
    


  dir.create(file.path(outdir, "deseq2_results"), showWarnings = FALSE)
  write.csv(res,
            file.path(outdir, "deseq2_results", paste0(tag, "_DESeq2.csv")),
            row.names = FALSE)
  write.csv(head(res, 50),
            file.path(outdir, "deseq2_results", paste0(tag, "_TOP50.csv")),
            row.names = FALSE)

  return(res)
}

counts dim: 62812 64 
meta dim: 64 5 

Unique fates:
[1] "FB"     "FB_HB"  "HB"     "HB_FB"  "MB"     "undiff"


# explore which timepoints exist for each fate (including FB_HB / HB_FB)

In [4]:
%%R
library(tidyr)

# --- Explore design without tibble printing (rpy2-safe) ---

cat("\nUnique celllines:\n")
print(sort(unique(meta$cellline)))

cat("\nUnique timepoints:\n")
print(sort(unique(meta$time)))

cat("\nUnique fates:\n")
print(sort(unique(meta$fate)))

cat("\nCounts: time x fate (pooled across lines):\n")
tf <- with(meta, table(time, fate))
print(tf)

cat("\nCounts: cellline x time x fate (as a flat table):\n")
ctf <- as.data.frame(with(meta, table(cellline, time, fate)))
ctf <- ctf[ctf$Freq > 0, ]
ctf <- ctf[order(ctf$cellline, ctf$time, ctf$fate), ]
print(ctf)

cat("\nSamples for crossover fates (FB_HB, HB_FB):\n")
xo <- meta[meta$fate %in% c("FB_HB","HB_FB"), ]
xo <- xo[order(xo$cellline, xo$time, xo$fate, xo$replicate), ]
print(xo)

# Save these tables to disk so you can inspect outside the notebook if needed
outdir <- "/home/kgr851/BULK_RNA"
write.csv(as.data.frame(tf), file.path(outdir, "design_time_x_fate.csv"), row.names=FALSE)
write.csv(ctf, file.path(outdir, "design_cellline_time_fate.csv"), row.names=FALSE)
write.csv(xo,  file.path(outdir, "design_crossover_samples.csv"), row.names=FALSE)

cat("\nWrote design tables to:", outdir, "\n")


Unique celllines:
[1] "H9"   "KOLF" "RC17"

Unique timepoints:
[1] "0h"  "12h" "24h" "3d"  "48h" "5d" 

Unique fates:
[1] "FB"     "FB_HB"  "HB"     "HB_FB"  "MB"     "undiff"

Counts: time x fate (pooled across lines):
     fate
time  FB FB_HB HB HB_FB MB undiff
  0h   0     0  0     0  0      2
  12h  2     0  2     0  2      0
  24h  2     0  2     0  2      0
  3d   2     0  2     0  2      0
  48h  6     0  6     0  6      0
  5d   6     6  6     6  2      0

Counts: cellline x time x fate (as a flat table):
   cellline time   fate Freq
91       H9   0h undiff    2
4        H9  12h     FB    2
40       H9  12h     HB    2
76       H9  12h     MB    2
7        H9  24h     FB    2
43       H9  24h     HB    2
79       H9  24h     MB    2
10       H9   3d     FB    2
46       H9   3d     HB    2
82       H9   3d     MB

    2
13       H9  48h     FB    2
49       H9  48h     HB    2
85       H9  48h     MB    2
16       H9   5d     FB    2
34       H9   5d  FB_HB    2
52       H9   5d     HB    2
70       H9   5d  HB_FB    2
88       H9   5d     MB    2
14     KOLF  48h     FB    2
50     KOLF  48h     HB    2
86     KOLF  48h     MB    2
17     KOLF   5d     FB    2
35     KOLF   5d  FB_HB    2
53     KOLF   5d     HB    2
71     KOLF   5d  HB_FB    2
15     RC17  48h     FB    2
51     RC17  48h     HB    2
87     RC17  48h     MB    2
18     RC17   5d     FB    2
36     RC17   5d  FB_HB    2
54     RC17   5d     HB    2
72     RC17   5d  HB_FB    2

Samples for crossover fates (FB_HB, HB_FB):
                               sample cellline time  fate replicate
H9_5d_FB_HB_Rep1     H9_5d_FB_HB_Rep1       H9   5d FB_HB         1
H9_5d_FB_HB_Rep2     H9_5d_FB_HB_Rep2       H9   5d FB_HB         2
H9_5d_HB_FB_Rep1     H9_5d_HB_FB_Rep1       H9   5d HB_FB         1
H9_5d_HB_FB_Rep2     H9_5d_HB_FB_Rep2  


Attaching package: ‘tidyr’

The following object is masked from ‘package:S4Vectors’:

    expand



# run fate-locking contrasts at 5d (with cellline in design)

In [5]:
%%R
suppressPackageStartupMessages({
  library(DESeq2)
  library(dplyr)
  library(tibble)
})

# --- Fate locking tests at 5d (pooled across H9/KOLF/RC17; adjust for cellline) ---

run_contrast_5d <- function(fateA, fateB, tag, min_total_counts = 10) {

  # Subset to ONLY these 2 fates at 5d  (=> MB is NOT included)
  keep <- meta$time == "5d" & meta$fate %in% c(fateA, fateB)
  meta2 <- meta[keep, , drop = FALSE]
  cts2  <- counts[, rownames(meta2), drop = FALSE]

  cat("\n---", tag, "---\n")
  cat("Fates:", fateA, "vs", fateB, "\n")
  cat("n samples:", ncol(cts2), "\n")
  print(table(meta2$cellline, meta2$fate))

  # Factors (clean + explicit reference)
  meta2$cellline <- droplevels(factor(meta2$cellline))
  meta2$fate     <- droplevels(factor(meta2$fate, levels = c(fateA, fateB)))

  # DESeq2
  dds <- DESeqDataSetFromMatrix(
    countData = cts2,
    colData   = meta2,
    design    = ~ cellline + fate
  )

  # Filter low-count genes
  dds <- dds[rowSums(counts(dds)) >= min_total_counts, ]

  dds <- DESeq(dds, quiet = TRUE)

  # Contrast: fateB relative to fateA (fateA is reference)
  #res <- results(dds, contrast = c("fate", fateB, fateA)) %>%
  res <- results(dds, contrast = c("fate", fateA, fateB)) %>%
    as.data.frame() %>%
    rownames_to_column("gene_id")

  # Sort with NAs last (more stable than arrange(padj) directly)
  res <- res %>%
    mutate(padj_sort = ifelse(is.na(padj), Inf, padj)) %>%
    arrange(padj_sort) %>%
    select(-padj_sort)

  # Write outputs
  out_subdir <- file.path(outdir, "deseq2_results_5d")
  dir.create(out_subdir, showWarnings = FALSE, recursive = TRUE)

  write.csv(res, file.path(out_subdir, paste0(tag, "_DESeq2.csv")), row.names = FALSE)
  write.csv(head(res, 50), file.path(out_subdir, paste0(tag, "_TOP50.csv")), row.names = FALSE)

  # Quick summary
  n_sig  <- sum(!is.na(res$padj) & res$padj < 0.05)
  n_sig2 <- sum(!is.na(res$padj) & res$padj < 0.05 & abs(res$log2FoldChange) > 1)

  cat("Sig (padj<0.05):", n_sig, "\n")
  cat("Sig (padj<0.05 & |log2FC|>1):", n_sig2, "\n")

  return(res)
}

# Core fate-locking contrasts (expect few DE genes):
res_FB_vs_FBHB <- run_contrast_5d("FB",    "FB_HB", "5d_FB_vs_FB_HB")
res_HB_vs_HBFB <- run_contrast_5d("HB",    "HB_FB", "5d_HB_vs_HB_FB")

# Anchor contrasts (expect many DE genes):
res_FB_vs_HB   <- run_contrast_5d("FB",    "HB",    "5d_FB_vs_HB")
res_FBHB_vs_HB <- run_contrast_5d("FB_HB", "HB",    "5d_FB_HB_vs_HB")
res_HBFB_vs_FB <- run_contrast_5d("HB_FB", "FB",    "5d_HB_FB_vs_FB")


--- 5d_FB_vs_FB_HB ---
Fates: FB vs FB_HB 
n samples: 12 
      
       FB FB_HB
  H9    2     2
  KOLF  2     2
  RC17  2     2


Sig (padj<0.05): 5704 
Sig (padj<0.05 & |log2FC|>1): 1862 

--- 5d_HB_vs_HB_FB ---
Fates: HB vs HB_FB 
n samples: 12 
      
       HB HB_FB
  H9    2     2
  KOLF  2     2
  RC17  2     2
Sig (padj<0.05): 6302 
Sig (padj<0.05 & |log2FC|>1): 1732 

--- 5d_FB_vs_HB ---
Fates: FB vs HB 
n samples: 12 
      
       FB HB
  H9    2  2
  KOLF  2  2
  RC17  2  2
Sig (padj<0.05): 8138 
Sig (padj<0.05 & |log2FC|>1): 2626 

--- 5d_FB_HB_vs_HB ---
Fates: FB_HB vs HB 
n samples: 12 
      
       FB_HB HB
  H9       2  2
  KOLF     2  2
  RC17     2  2
Sig (padj<0.05): 3533 
Sig (padj<0.05 & |log2FC|>1): 1337 

--- 5d_HB_FB_vs_FB ---
Fates: HB_FB vs FB 
n samples: 12 
      
       FB HB_FB
  H9    2     2
  KOLF  2     2
  RC17  2     2
Sig (padj<0.05): 6563 
Sig (padj<0.05 & |log2FC|>1): 2003 


# run the crossover-vs-crossover DESeq2 (FB_HB vs HB_FB) at 5d

In [6]:
%%R
suppressPackageStartupMessages({
  library(DESeq2)
  library(dplyr)
  library(tibble)
})

run_contrast_5d <- function(fateA, fateB, tag, min_total_counts = 10) {

  keep <- meta$time == "5d" & meta$fate %in% c(fateA, fateB)
  meta2 <- meta[keep, , drop = FALSE]
  cts2  <- counts[, rownames(meta2), drop = FALSE]

  cat("\n---", tag, "---\n")
  cat("Fates:", fateA, "vs", fateB, "\n")
  cat("n samples:", ncol(cts2), "\n")
  print(table(meta2$cellline, meta2$fate))

  meta2$cellline <- droplevels(factor(meta2$cellline))
  meta2$fate     <- droplevels(factor(meta2$fate, levels = c(fateA, fateB)))

  dds <- DESeqDataSetFromMatrix(
    countData = cts2,
    colData   = meta2,
    design    = ~ cellline + fate
  )

  dds <- dds[rowSums(counts(dds)) >= min_total_counts, ]
  dds <- DESeq(dds, quiet = TRUE)

  #res <- results(dds, contrast = c("fate", fateB, fateA)) %>%
  res <- results(dds, contrast = c("fate", fateA, fateB)) %>%
    as.data.frame() %>%
    rownames_to_column("gene_id")

  res <- res %>%
    mutate(padj_sort = ifelse(is.na(padj), Inf, padj)) %>%
    arrange(padj_sort) %>%
    select(-padj_sort)

  out_subdir <- file.path(outdir, "deseq2_results_5d")
  dir.create(out_subdir, showWarnings = FALSE, recursive = TRUE)

  write.csv(res, file.path(out_subdir, paste0(tag, "_DESeq2.csv")), row.names = FALSE)

  cat("Sig (padj<0.05):", sum(!is.na(res$padj) & res$padj < 0.05), "\n")
  cat("Sig (padj<0.05 & |log2FC|>1):", sum(!is.na(res$padj) & res$padj < 0.05 & abs(res$log2FoldChange) > 1), "\n")

  invisible(res)
}

# You already have 5d_FB_vs_HB_DESeq2.csv from earlier.
# Now generate crossover-vs-crossover:
run_contrast_5d("FB_HB", "HB_FB", "5d_FB_HB_vs_HB_FB")


--- 5d_FB_HB_vs_HB_FB ---
Fates: FB_HB vs HB_FB 
n samples: 12 
      
       FB_HB HB_FB
  H9       2     2
  KOLF     2     2
  RC17     2     2


Sig (padj<0.05): 6631 
Sig (padj<0.05 & |log2FC|>1): 2457 


## same thing independet 3 lines

In [7]:
%%R
library(DESeq2)
library(dplyr)
library(tibble)

outdir <- "/home/kgr851/BULK_RNA"
resdir <- file.path(outdir, "deseq2_results_5d_by_cellline")
dir.create(resdir, showWarnings = FALSE)

run_contrast_5d_by_line <- function(cellline_name, fateA, fateB) {

  keep <- meta$time == "5d" &
          meta$cellline == cellline_name &
          meta$fate %in% c(fateA, fateB)

  meta2 <- meta[keep, , drop=FALSE]
  cts2  <- counts[, rownames(meta2), drop=FALSE]

  cat("\n---", cellline_name, ":", fateA, "vs", fateB, "---\n")
  print(table(meta2$fate))

  if (ncol(cts2) < 4) {
    cat("Skipping (not enough samples)\n")
    return(NULL)
  }

  meta2$fate <- factor(meta2$fate)
  meta2$fate <- relevel(meta2$fate, ref=fateA)

  dds <- DESeqDataSetFromMatrix(
    countData = cts2,
    colData   = meta2,
    design    = ~ fate
  )

  dds <- dds[rowSums(counts(dds)) >= 10, ]
  dds <- DESeq(dds, quiet = TRUE)

  #res <- results(dds, contrast = c("fate", fateB, fateA)) %>%
   res <- results(dds, contrast = c("fate", fateA, fateB)) %>%
    as.data.frame() %>%
    rownames_to_column("gene_id") %>%
    arrange(padj)

  fname <- paste0(
    "DESeq2_RNA_",
    cellline_name,
    "_5d_",
    fateA,
    "_vs_",
    fateB,
    ".csv"
  )

  write.csv(res, file.path(resdir, fname), row.names = FALSE)

  cat("Saved:", fname, "\n")
  cat("Sig (padj<0.05):",
      sum(!is.na(res$padj) & res$padj < 0.05), "\n")
}

celllines <- c("H9", "KOLF", "RC17")

for (cl in celllines) {
  # anchor
  run_contrast_5d_by_line(cl, "FB",    "HB")
  # crossover test
  run_contrast_5d_by_line(cl, "FB_HB", "HB_FB")
}

cat("\nAll per-line DESeq2 results written to:\n", resdir, "\n")


--- H9 : FB vs HB ---

FB HB 
 2  2 


Saved: DESeq2_RNA_H9_5d_FB_vs_HB.csv 
Sig (padj<0.05): 4490 

--- H9 : FB_HB vs HB_FB ---

FB_HB HB_FB 
    2     2 
Saved: DESeq2_RNA_H9_5d_FB_HB_vs_HB_FB.csv 
Sig (padj<0.05): 1864 

--- KOLF : FB vs HB ---

FB HB 
 2  2 
Saved: DESeq2_RNA_KOLF_5d_FB_vs_HB.csv 
Sig (padj<0.05): 5580 

--- KOLF : FB_HB vs HB_FB ---

FB_HB HB_FB 
    2     2 
Saved: DESeq2_RNA_KOLF_5d_FB_HB_vs_HB_FB.csv 
Sig (padj<0.05): 4475 

--- RC17 : FB vs HB ---

FB HB 
 2  2 
Saved: DESeq2_RNA_RC17_5d_FB_vs_HB.csv 
Sig (padj<0.05): 6283 

--- RC17 : FB_HB vs HB_FB ---

FB_HB HB_FB 
    2     2 
Saved: DESeq2_RNA_RC17_5d_FB_HB_vs_HB_FB.csv 
Sig (padj<0.05): 4288 

All per-line DESeq2 results written to:
 /home/kgr851/BULK_RNA/deseq2_results_5d_by_cellline 


In [9]:
%%R
library(DESeq2)
library(dplyr)
library(tibble)

outdir <- "/home/kgr851/BULK_RNA"

# ============================================================
# 1) Per-cellline: FB vs HB, FB vs FB_HB, HB vs HB_FB
# ============================================================
resdir_by_line <- file.path(outdir, "deseq2_results_5d_by_cellline_all_contrasts")
dir.create(resdir_by_line, showWarnings = FALSE, recursive = TRUE)

run_contrast_per_line <- function(cellline_name, fateA, fateB) {
  keep <- meta$time == "5d" &
          meta$cellline == cellline_name &
          meta$fate %in% c(fateA, fateB)

  meta2 <- meta[keep, , drop = FALSE]
  cts2  <- counts[, rownames(meta2), drop = FALSE]

  tag <- paste0(cellline_name, "_5d_", fateA, "_vs_", fateB)
  cat("\n---", tag, "---\n")
  cat("Samples per fate:\n")
  print(table(meta2$fate))
  cat("Total n samples:", ncol(cts2), "\n")

  if (ncol(cts2) < 2) {
    cat("Skipping (not enough samples)\n")
    return(NULL)
  }

  meta2$fate <- factor(meta2$fate)
  meta2$fate <- relevel(meta2$fate, ref = fateA)

  dds <- DESeqDataSetFromMatrix(
    countData = cts2,
    colData   = meta2,
    design    = ~ fate
  )

  dds <- dds[rowSums(counts(dds)) >= 10, ]
  dds <- DESeq(dds, quiet = TRUE)

  res <- results(dds, contrast = c("fate", fateA, fateB)) %>%
    as.data.frame() %>%
    rownames_to_column("gene_id") %>%
    arrange(padj)

  fname <- paste0(tag, "_DESeq2.csv")
  write.csv(res, file.path(resdir_by_line, fname), row.names = FALSE)

  cat("Saved:", fname, "\n")
  n_sig <- sum(!is.na(res$padj) & res$padj < 0.05)
  cat("Sig (padj<0.05):", n_sig, "\n")

  invisible(res)
}

celllines <- c("H9", "RC17", "KOLF")

cat("\n========== PER-CELLLINE CONTRASTS ==========\n")
for (cl in celllines) {
  run_contrast_per_line(cl, "FB", "HB")
  run_contrast_per_line(cl, "FB", "FB_HB")
  run_contrast_per_line(cl, "HB", "HB_FB")
}

# ============================================================
# 2) Pooled (all celllines together): FB vs HB, FB vs FB_HB, HB vs HB_FB
# ============================================================
resdir_pooled <- file.path(outdir, "deseq2_results_5d_pooled")
dir.create(resdir_pooled, showWarnings = FALSE, recursive = TRUE)

run_contrast_pooled <- function(fateA, fateB) {
  keep <- meta$time == "5d" &
          meta$fate %in% c(fateA, fateB)

  meta2 <- meta[keep, , drop = FALSE]
  cts2  <- counts[, rownames(meta2), drop = FALSE]

  tag <- paste0("5d_pooled_", fateA, "_vs_", fateB)
  cat("\n---", tag, "---\n")
  cat("Celllines and fates:\n")
  print(table(meta2$cellline, meta2$fate))
  cat("Total n samples:", ncol(cts2), "\n")

  if (ncol(cts2) < 2) {
    cat("Skipping (not enough samples)\n")
    return(NULL)
  }

  meta2$cellline <- droplevels(factor(meta2$cellline))
  meta2$fate     <- droplevels(factor(meta2$fate, levels = c(fateA, fateB)))

  dds <- DESeqDataSetFromMatrix(
    countData = cts2,
    colData   = meta2,
    design    = ~ cellline + fate
  )

  dds <- dds[rowSums(counts(dds)) >= 10, ]
  dds <- DESeq(dds, quiet = TRUE)

  res <- results(dds, contrast = c("fate", fateA, fateB)) %>%
    as.data.frame() %>%
    rownames_to_column("gene_id") %>%
    mutate(padj_sort = ifelse(is.na(padj), Inf, padj)) %>%
    arrange(padj_sort) %>%
    select(-padj_sort)

  fname <- paste0(tag, "_DESeq2.csv")
  write.csv(res, file.path(resdir_pooled, fname), row.names = FALSE)

  cat("Saved:", fname, "\n")
  n_sig <- sum(!is.na(res$padj) & res$padj < 0.05)
  cat("Sig (padj<0.05):", n_sig, "\n")

  invisible(res)
}

cat("\n========== POOLED CONTRASTS (all celllines) ==========\n")
run_contrast_pooled("FB", "HB")
run_contrast_pooled("FB", "FB_HB")
run_contrast_pooled("HB", "HB_FB")

cat("\nAll DESeq2 results written to:\n")
cat("  Per-line:", resdir_by_line, "\n")
cat("  Pooled:  ", resdir_pooled, "\n")



========== PER-CELLLINE CONTRASTS ==========

--- H9_5d_FB_vs_HB ---
Samples per fate:

FB HB 
 2  2 
Total n samples: 4 
Saved: H9_5d_FB_vs_HB_DESeq2.csv 
Sig (padj<0.05): 4490 

--- H9_5d_FB_vs_FB_HB ---
Samples per fate:

   FB FB_HB 
    2     2 
Total n samples: 4 
Saved: H9_5d_FB_vs_FB_HB_DESeq2.csv 
Sig (padj<0.05): 1204 

--- H9_5d_HB_vs_HB_FB ---
Samples per fate:

   HB HB_FB 
    2     2 
Total n samples: 4 
Saved: H9_5d_HB_vs_HB_FB_DESeq2.csv 
Sig (padj<0.05): 2672 

--- RC17_5d_FB_vs_HB ---
Samples per fate:

FB HB 
 2  2 
Total n samples: 4 
Saved: RC17_5d_FB_vs_HB_DESeq2.csv 
Sig (padj<0.05): 6283 

--- RC17_5d_FB_vs_FB_HB ---
Samples per fate:

   FB FB_HB 
    2     2 
Total n samples: 4 
Saved: RC17_5d_FB_vs_FB_HB_DESeq2.csv 
Sig (padj<0.05): 4298 

--- RC17_5d_HB_vs_HB_FB ---
Samples per fate:

   HB HB_FB 
    2     2 
Total n samples: 4 
Saved: RC17_5d_HB_vs_HB_FB_DESeq2.csv 
Sig (padj<0.05): 6676 

--- KOLF_5d_FB_vs_HB ---
Samples per fate:

FB HB 
 2  2 
Total n

In [10]:
import pandas as pd
import numpy as np
from pathlib import Path

# ============================================================
# Export raw 5d counts to Excel (for bar graphs)
# ============================================================

BASE = Path("/home/kgr851/BULK_RNA")

# Load metadata and counts
meta_path = BASE / "RNA_metadata_all.csv"
counts_path = BASE / "RNA_counts_all_INT.csv"
gene_map_path = BASE / "gene_id_to_name.csv"

print("Loading data...")
meta = pd.read_csv(meta_path)
counts = pd.read_csv(counts_path, index_col=0)
gene_map = pd.read_csv(gene_map_path)

# Ensure gene_id is string for merging
gene_map["gene_id"] = gene_map["gene_id"].astype(str)
gene_map = gene_map.drop_duplicates(subset=["gene_id"], keep="first")

# Filter to 5d samples only
meta_5d = meta[meta["time"] == "5d"].copy()
samples_5d = meta_5d["sample"].tolist()

print(f"Found {len(samples_5d)} samples at 5d")
print(f"Counts shape: {counts.shape}")

# Subset counts to 5d samples
counts_5d = counts[samples_5d].copy()
counts_5d.index.name = "gene_id"
counts_5d = counts_5d.reset_index()

# Add gene names
counts_5d["gene_id"] = counts_5d["gene_id"].astype(str)
counts_5d = counts_5d.merge(
    gene_map[["gene_id", "gene_name"]], 
    on="gene_id", 
    how="left"
)

# Reorder: gene_id, gene_name, then samples
gene_cols = ["gene_id", "gene_name"]
sample_cols = [col for col in counts_5d.columns if col not in gene_cols]
counts_5d = counts_5d[gene_cols + sample_cols]

print(f"\n5d counts shape: {counts_5d.shape}")
print(f"Columns: {list(counts_5d.columns[:10])}...")

# Create Excel file with multiple sheets:
# Sheet 1: All counts with metadata in header
# Sheet 2: Sample metadata for reference
output_file = BASE / "RNA_5d_counts_ALL_genes_for_bargraphs.xlsx"

print(f"\nWriting to: {output_file}")

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    # Sheet 1: counts (all genes, all 5d samples)
    counts_5d.to_excel(writer, sheet_name="5d_raw_counts", index=False)
    
    # Sheet 2: metadata for the 5d samples
    meta_5d_sorted = meta_5d.set_index("sample").loc[samples_5d].reset_index()
    meta_5d_sorted.to_excel(writer, sheet_name="5d_sample_metadata", index=False)
    
    # Sheet 3: Gene map reference
    gene_map.to_excel(writer, sheet_name="gene_id_to_name_reference", index=False)

print(f"\n✓ Excel file created successfully!")
print(f"\nSummary of 5d samples:")
print(meta_5d.groupby(["cellline", "fate"]).size().unstack(fill_value=0))

print(f"\nFile: {output_file}")
print(f"Sheets:")
print(f"  1. '5d_raw_counts' - {counts_5d.shape[0]} genes × {len(samples_5d)} samples")
print(f"  2. '5d_sample_metadata' - Sample info (cellline, fate, replicate, etc)")
print(f"  3. 'gene_id_to_name_reference' - Gene ID ↔ Gene name mapping")


Loading data...
Found 26 samples at 5d
Counts shape: (62812, 64)

5d counts shape: (62812, 28)
Columns: ['gene_id', 'gene_name', 'H9_5d_FB_HB_Rep1', 'H9_5d_FB_HB_Rep2', 'H9_5d_FB_Rep1', 'H9_5d_FB_Rep2', 'H9_5d_HB_FB_Rep1', 'H9_5d_HB_FB_Rep2', 'H9_5d_HB_Rep1', 'H9_5d_HB_Rep2']...

Writing to: /home/kgr851/BULK_RNA/RNA_5d_counts_ALL_genes_for_bargraphs.xlsx

✓ Excel file created successfully!

Summary of 5d samples:
fate      FB  FB_HB  HB  HB_FB  MB
cellline                          
H9         2      2   2      2   2
KOLF       2      2   2      2   0
RC17       2      2   2      2   0

File: /home/kgr851/BULK_RNA/RNA_5d_counts_ALL_genes_for_bargraphs.xlsx
Sheets:
  1. '5d_raw_counts' - 62812 genes × 26 samples
  2. '5d_sample_metadata' - Sample info (cellline, fate, replicate, etc)
  3. 'gene_id_to_name_reference' - Gene ID ↔ Gene name mapping


In [4]:
import pandas as pd
from pathlib import Path

# Paths
BASE = Path("/home/kgr851/BULK_RNA")
meta_path = BASE / "RNA_metadata_all.csv"
counts_path = BASE / "RNA_counts_all_INT.csv"
gene_map_path = BASE / "gene_id_to_name.csv"

# Timepoints to include (in order)
TIME_ORDER = ["0h", "12h", "24h", "48h", "3d", "5d"]

# Load
meta = pd.read_csv(meta_path)
counts = pd.read_csv(counts_path, index_col=0)
gene_map = pd.read_csv(gene_map_path)

# Clean gene map
gene_map["gene_id"] = gene_map["gene_id"].astype(str)
gene_map = gene_map.drop_duplicates("gene_id")[["gene_id", "gene_name"]]

# Keep only requested timepoints and order samples
meta_sel = meta[meta["time"].astype(str).isin(TIME_ORDER)].copy()
meta_sel["time"] = pd.Categorical(meta_sel["time"].astype(str), categories=TIME_ORDER, ordered=True)
meta_sel = meta_sel.sort_values(["time", "cellline", "fate", "replicate", "sample"])

# Sample list present in counts
samples = [s for s in meta_sel["sample"].astype(str).tolist() if s in counts.columns]
if len(samples) == 0:
    raise ValueError("No matching samples found in counts for selected timepoints.")

# Build counts table: gene_id, gene_name, then all selected samples
out_df = counts[samples].copy()
out_df.index.name = "gene_id"
out_df = out_df.reset_index()
out_df["gene_id"] = out_df["gene_id"].astype(str)
out_df = out_df.merge(gene_map, on="gene_id", how="left")
out_df = out_df[["gene_id", "gene_name"] + samples]

# Write ONE excel sheet
out_xlsx = BASE / "RNA_raw_counts_0h_12h_24h_48h_3d_5d_one_sheet.xlsx"
with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
    out_df.to_excel(writer, sheet_name="raw_counts_all_timepoints", index=False)

print("Done.")
print(f"Wrote: {out_xlsx}")
print(f"Shape: {out_df.shape[0]} genes x {len(samples)} samples")


Done.
Wrote: /home/kgr851/BULK_RNA/RNA_raw_counts_0h_12h_24h_48h_3d_5d_one_sheet.xlsx
Shape: 62812 genes x 64 samples


In [ ]:
#!jupyter nbconvert --to html bulk_rna_v3.ipynb

[NbConvertApp] Converting notebook bulk_rna_v3.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 7 image(s).
[NbConvertApp] Writing 3417880 bytes to bulk_rna_v3.html
